In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd
import tifffile as tiff

from fimpylab import LightsheetExperiment

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.pca import pca_and_phase, get_fictive_heading, fictive_heading_and_fit, \
        fit_phase_neurons,qap_sorting_and_phase
from circle_fit import hyper_fit
from lotr.experiment_class import LotrExperiment
import json

from lotr.plotting.color_utils import get_n_colors
from scipy.interpolate import interp1d
from scipy import signal

In [ ]:
path = Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring\210314_f1\210314_f1_natmov")
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files = list(master.glob("*/*_f*"))
path = files[71] 
print(path)

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr").T
try:
    selected = fl.load(path / "selected.h5")
except:
    print("no selected")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
try:
    coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
    anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
except:
    coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
    anat = fl.load(path / "data_from_suite2p_cells.h5", "/anatomy_stack")

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 150
t_lims = [150, -50]
t_slice = slice(*t_lims)

In [ ]:
print(np.shape(traces))
print(selected)

In [ ]:
stim_log = exp.stimulus_log
beh_log = exp.behavior_log
stim_log

In [ ]:
try:
    theta = np.asarray(stim_log['bg_theta'])
except:
    theta1 = np.asarray(stim_log['open_loop_theta'])
    theta1[np.where(np.isnan(theta1))[0]] = 0
    theta2 = np.asarray(stim_log['closed_loop_theta'])
    theta2[np.where(np.isnan(theta2))[0]] = 0
    theta = theta1 + theta2
#theta = np.asarray(stim_log['cl2D_vel'])
stim_t = np.asarray(stim_log.t)

theta_wrapped = np.mod(theta, np.pi * 2) - np.pi
theta_diff = np.zeros_like(theta)
theta_diff[1:] = np.diff(theta)
theta_diff[theta_diff < -0.5] = 0
theta_diff *= 50

pos_theta = np.copy(theta_diff)
pos_theta[pos_theta < 0] = 0

neg_theta = np.copy(theta_diff)
neg_theta[neg_theta > 0] = 0

theta_off = np.zeros_like(theta)
theta_off[1:] = np.diff(theta)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(stim_t, (theta / 5)+ 5)
ax.plot(stim_t, theta_wrapped)
ax.plot(stim_t, (theta_diff)-10)
ax.plot(beh_log.t, beh_log.tail_sum - 30)


In [ ]:
fs = exp.fn
print(fs)
dt_imaging = 1 / fs
int_fact = 200
t_imaging = np.arange(traces.shape[1])/fs
num_traces, len_rec = np.shape(traces)

# generating regressors
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS

tau_fs = REGRESSOR_TAU_S * fn
kernel = np.exp(-np.arange(1000) / tau_fs)
t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact

In [ ]:
theta_int = interp1d(stim_log.t, theta, bounds_error=False, fill_value=0)(t_imaging_int)
theta_reg = signal.decimate(theta_int, int_fact, ftype="fir")
theta_reg_conv = np.convolve(theta_reg, kernel)[:np.shape(traces)[1]]

theta_wrp_int = interp1d(stim_log.t, theta_wrapped, bounds_error=False, fill_value=0)(t_imaging_int)
theta_wrp_reg = signal.decimate(theta_wrp_int, int_fact, ftype="fir")
theta_wrp_conv = np.convolve(theta_wrp_reg, kernel)[:np.shape(traces)[1]]

theta_diff_int = interp1d(stim_log.t, theta_diff, bounds_error=False, fill_value=0)(t_imaging_int)
theta_diff_reg = signal.decimate(theta_diff_int, int_fact, ftype="fir")
theta_diff_conv = np.convolve(theta_diff_reg, kernel)[:np.shape(traces)[1]]

theta_off_int = interp1d(stim_log.t, theta_off, bounds_error=False, fill_value=0)(t_imaging_int)
theta_off_reg = signal.decimate(theta_off_int, int_fact, ftype="fir") * -1
theta_off_conv = np.convolve(theta_off_reg, kernel)[:np.shape(traces)[1]]


In [ ]:
np.shape(traces)

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(6, 7))
stim_t_conv = np.arange(np.shape(theta_reg_conv)[0]) / 5
ax[0].plot(stim_t_conv, theta_reg_conv / 5 + 150)
ax[0].plot(stim_t_conv, theta_wrp_conv)
ax[0].plot(stim_t_conv, theta_diff_conv + 80)
ax[0].plot(beh_log.t, beh_log.tail_sum*10 - 80)
ax[0].plot(stim_t_conv, theta_off_conv/5 - 200)
ax[0].set_xlim(0, stim_t_conv[-1])

ax[1].imshow(traces, vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
ax[0].axis('off')
ax[1].axis('off')

ax[2].imshow(traces[selected], vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
ax[2].axis('off')
file_name = 'theta_regressors.jpg'
fig.savefig(path / file_name)

In [ ]:
theta_traces = np.dot(traces, theta_reg_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_reg_conv)
theta_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_reg_conv)

theta_wrp_traces = np.dot(traces, theta_wrp_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_wrp_conv)
theta_wrp_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_wrp_conv)

theta_diff_traces = np.dot(traces, theta_diff_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
theta_diff_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_diff_conv)

theta_off_traces = np.dot(traces, theta_off_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_off_conv)
theta_off_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_off_conv)

In [ ]:
reg_values = {
    'theta_reg': theta_traces,
    'theta_wrp_reg': theta_wrp_traces,
    'theta_diff_reg': theta_diff_traces,
    'theta_off_reg': theta_off_traces,
    }
fl.save(str(path / 'stimulus_regression_values.h5'), reg_values)

In [ ]:
fig_anatomy2, ax_anatomy2 = plt.subplots(2, 3, figsize=(7, 6), sharex=True, sharey=True)

ax_anatomy2[0,0].scatter(coords[:, 1], coords[:, 2], c=theta_wrp_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[0,0].axis('off')
ax_anatomy2[0,0].set_title('Theta wrapped')

ax_anatomy2[0,1].scatter(coords[:, 1], coords[:, 2], c=theta_diff_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[0,1].axis('off')
ax_anatomy2[0,1].set_title('Theta diff')

ax_anatomy2[0,2].scatter(coords[:, 1], coords[:, 2], c=theta_off_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[0,2].axis('off')
ax_anatomy2[0,2].set_title('Theta off')

ax_anatomy2[1,0].scatter(coords[selected, 1], coords[selected, 2], c=theta_wrp_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[1,0].axis('off')

ax_anatomy2[1,1].scatter(coords[selected, 1], coords[selected, 2], c=theta_diff_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[1,1].axis('off')

ax_anatomy2[1,2].scatter(coords[selected, 1], coords[selected, 2], c=theta_off_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[1,2].axis('off')

file_name = 'theta_regressors_rois.jpg'
fig_anatomy2.savefig(path / file_name)


In [ ]:
# making one big figure:
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files = list(master.glob("*/*_f*_natmov*"))
print(len(files))

In [ ]:
fig_anatomy2, ax_anatomy2 = plt.subplots(7, 6, figsize=(10, 10), sharex=True, sharey=True)

count = 0
for i in range(len(files)):
    path = files[i]
    traces = fl.load(path / "filtered_traces.h5", "/detr").T
    
    try:
        with open(next(path.glob("*metadata.json"))) as gg:
            metadata = json.load(gg)
        fish_id = metadata['general']['fish_id']
    except:
        fish_id = ""
    print(fish_id)
    try:
        selected = fl.load(path / "selected.h5")
        
        reg_df = fl.load(path / "motor_regressors.h5")
        cc_motor = reg_df["all_bias_abs"].values
        cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
        try:
            coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
            anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
        except:
            coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
            anat = fl.load(path / "data_from_suite2p_cells.h5", "/anatomy_stack")

        df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
        exp = LotrExperiment(path)
        fn = int(exp.fn)

        t_start_s = 150
        t_lims = [150, -50]
        t_slice = slice(*t_lims)
        
        stim_log = exp.stimulus_log
        beh_log = exp.behavior_log
    
        try:
            theta = np.asarray(stim_log['bg_theta'])
        except:
            theta1 = np.asarray(stim_log['open_loop_theta'])
            theta1[np.where(np.isnan(theta1))[0]] = 0
            theta2 = np.asarray(stim_log['closed_loop_theta'])
            theta2[np.where(np.isnan(theta2))[0]] = 0
            theta = theta1 + theta2
        #theta = np.asarray(stim_log['cl2D_vel'])
        stim_t = np.asarray(stim_log.t)

        theta_wrapped = np.mod(theta, np.pi * 2) - np.pi
        theta_diff = np.zeros_like(theta)
        theta_diff[1:] = np.diff(theta)
        theta_diff[theta_diff < -0.5] = 0
        theta_diff *= 50

        pos_theta = np.copy(theta_diff)
        pos_theta[pos_theta < 0] = 0

        neg_theta = np.copy(theta_diff)
        neg_theta[neg_theta > 0] = 0

        theta_off = np.zeros_like(theta)
        theta_off[1:] = np.diff(theta)
        
        fs = exp.fn
        dt_imaging = 1 / fs
        int_fact = 200
        t_imaging = np.arange(traces.shape[1])/fs
        num_traces, len_rec = np.shape(traces)

        # generating regressors
        tau_fs = REGRESSOR_TAU_S * fn
        kernel = np.exp(-np.arange(1000) / tau_fs)
        t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact
    
        theta_int = interp1d(stim_log.t, theta, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_reg = signal.decimate(theta_int, int_fact, ftype="fir")
        theta_reg_conv = np.convolve(theta_reg, kernel)[:np.shape(traces)[1]]

        theta_wrp_int = interp1d(stim_log.t, theta_wrapped, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_wrp_reg = signal.decimate(theta_wrp_int, int_fact, ftype="fir")
        theta_wrp_conv = np.convolve(theta_wrp_reg, kernel)[:np.shape(traces)[1]]

        theta_diff_int = interp1d(stim_log.t, theta_diff, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_diff_reg = signal.decimate(theta_diff_int, int_fact, ftype="fir")
        theta_diff_conv = np.convolve(theta_diff_reg, kernel)[:np.shape(traces)[1]]

        theta_off_int = interp1d(stim_log.t, theta_off, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_off_reg = signal.decimate(theta_off_int, int_fact, ftype="fir") * -1
        theta_off_conv = np.convolve(theta_off_reg, kernel)[:np.shape(traces)[1]]
        
        
        fig, ax = plt.subplots(3,1, figsize=(6, 7))
        stim_t_conv = np.arange(np.shape(theta_reg_conv)[0]) / 5
        ax[0].plot(stim_t_conv, theta_reg_conv / 5 + 150)
        ax[0].plot(stim_t_conv, theta_wrp_conv)
        ax[0].plot(stim_t_conv, theta_diff_conv + 80)
        ax[0].plot(beh_log.t, beh_log.tail_sum*10 - 80)
        ax[0].plot(stim_t_conv, theta_off_conv/5 - 200)
        ax[0].set_xlim(0, stim_t_conv[-1])

        ax[1].imshow(traces, vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
        ax[0].axis('off')
        ax[1].axis('off')

        ax[2].imshow(traces[selected], vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
        ax[2].axis('off')
        file_name = 'theta_regressors.jpg'
        fig.savefig(path / file_name)

        theta_traces = np.dot(traces, theta_reg_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_reg_conv)
        theta_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_reg_conv)

        theta_wrp_traces = np.dot(traces, theta_wrp_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_wrp_conv)
        theta_wrp_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_wrp_conv)

        theta_diff_traces = np.dot(traces, theta_diff_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
        theta_diff_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_diff_conv)

        theta_off_traces = np.dot(traces, theta_off_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
        theta_off_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_off_conv)
        
        

        ax_anatomy2[i-count,0].scatter(coords[:, 1], coords[:, 2], c=theta_wrp_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        #ax_anatomy2[i-count,0].axis('off')
        ax_anatomy2[i-count,0].spines['right'].set_visible(False)
        ax_anatomy2[i-count,0].spines['top'].set_visible(False)
        ax_anatomy2[i-count,0].spines['left'].set_visible(False)
        ax_anatomy2[i-count,0].spines['bottom'].set_visible(False)
        ax_anatomy2[i-count,0].set_xticks([])
        ax_anatomy2[i-count,0].set_yticks([])
        ax_anatomy2[i-count,0].set_ylabel(fish_id)
        
        ax_anatomy2[i-count,1].scatter(coords[:, 1], coords[:, 2], c=theta_diff_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,1].axis('off')

        ax_anatomy2[i-count,2].scatter(coords[:, 1], coords[:, 2], c=theta_off_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,2].axis('off')
        
        ax_anatomy2[i-count,3].scatter(coords[selected, 1], coords[selected, 2], c=theta_wrp_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,3].axis('off')
        
        ax_anatomy2[i-count,4].scatter(coords[selected, 1], coords[selected, 2], c=theta_diff_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,4].axis('off')

        ax_anatomy2[i-count,5].scatter(coords[selected, 1], coords[selected, 2], c=theta_off_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,5].axis('off')

        
    except:
        print("no selected")
        count += 1
        
ax_anatomy2[0,0].set_title('Theta position')
ax_anatomy2[0,1].set_title('Theta diff')
ax_anatomy2[0,2].set_title('Theta off')

ax_anatomy2[0,3].set_title('Theta position - selected')
ax_anatomy2[0,4].set_title('Theta diff - selected')
ax_anatomy2[0,5].set_title('Theta off - selected')

file_name = 'theta_regressors_rois_natmov.jpg'
fig_anatomy2.savefig(master / file_name, dpi=300)


In [ ]:
# making one big figure:
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files = list(master.glob("*/*_f*_clol*"))
print(len(files))

In [ ]:
fig_anatomy2, ax_anatomy2 = plt.subplots(7, 6, figsize=(10, 10), sharex=True, sharey=True)
fig_anatomy1, ax_anatomy1 = plt.subplots(8, 1, figsize=(6, 12))


count = 0
for i in range(len(files)):
    path = files[i]
    traces = fl.load(path / "filtered_traces.h5", "/detr").T
    
    try:
        with open(next(path.glob("*metadata.json"))) as gg:
            metadata = json.load(gg)
        fish_id = metadata['general']['fish_id']
    except:
        fish_id = ""
    print(fish_id)
    
    try:
        selected = fl.load(path / "selected.h5")
        
        reg_df = fl.load(path / "motor_regressors.h5")
        cc_motor = reg_df["all_bias_abs"].values
        cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
        try:
            coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
            anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
        except:
            coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
            anat = fl.load(path / "data_from_suite2p_cells.h5", "/anatomy_stack")

        df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
        exp = LotrExperiment(path)
        fn = int(exp.fn)

        t_start_s = 150
        t_lims = [150, -50]
        t_slice = slice(*t_lims)
        
        stim_log = exp.stimulus_log
        beh_log = exp.behavior_log
    
        try:
            theta = np.asarray(stim_log['bg_theta'])
        except:
            theta1 = np.asarray(stim_log['open_loop_theta'])
            theta1[np.where(np.isnan(theta1))[0]] = 0
            theta2 = np.asarray(stim_log['closed_loop_theta'])
            theta2[np.where(np.isnan(theta2))[0]] = 0
            theta = theta1 + theta2
        #theta = np.asarray(stim_log['cl2D_vel'])
        stim_t = np.asarray(stim_log.t)

        theta_wrapped = np.mod(theta, np.pi * 2) - np.pi
        theta_diff = np.zeros_like(theta)
        theta_diff[1:] = np.diff(theta)
        theta_diff[theta_diff < -0.5] = 0
        theta_diff *= 50

        pos_theta = np.copy(theta_diff)
        pos_theta[pos_theta < 0] = 0

        neg_theta = np.copy(theta_diff)
        neg_theta[neg_theta > 0] = 0

        theta_off = np.zeros_like(theta)
        theta_off[1:] = np.diff(theta)
        
        fs = exp.fn
        dt_imaging = 1 / fs
        int_fact = 200
        t_imaging = np.arange(traces.shape[1])/fs
        num_traces, len_rec = np.shape(traces)

        # generating regressors
        tau_fs = REGRESSOR_TAU_S * fn
        kernel = np.exp(-np.arange(1000) / tau_fs)
        t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact
    
        theta_int = interp1d(stim_log.t, theta, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_reg = signal.decimate(theta_int, int_fact, ftype="fir")
        theta_reg_conv = np.convolve(theta_reg, kernel)[:np.shape(traces)[1]]

        theta_wrp_int = interp1d(stim_log.t, theta_wrapped, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_wrp_reg = signal.decimate(theta_wrp_int, int_fact, ftype="fir")
        theta_wrp_conv = np.convolve(theta_wrp_reg, kernel)[:np.shape(traces)[1]]

        theta_diff_int = interp1d(stim_log.t, theta_diff, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_diff_reg = signal.decimate(theta_diff_int, int_fact, ftype="fir")
        theta_diff_conv = np.convolve(theta_diff_reg, kernel)[:np.shape(traces)[1]]

        theta_off_int = interp1d(stim_log.t, theta_off, bounds_error=False, fill_value=0)(t_imaging_int)
        theta_off_reg = signal.decimate(theta_off_int, int_fact, ftype="fir") * -1
        theta_off_conv = np.convolve(theta_off_reg, kernel)[:np.shape(traces)[1]]
        
        
        stim_t_conv = np.arange(np.shape(theta_wrp_conv)[0]) / 5

        ax_anatomy1[i - count + 1].imshow(traces[selected], vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
        ax_anatomy1[i - count + 1].axis('off')

        theta_traces = np.dot(traces, theta_reg_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_reg_conv)
        theta_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_reg_conv)

        theta_wrp_traces = np.dot(traces, theta_wrp_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_wrp_conv)
        theta_wrp_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_wrp_conv)

        theta_diff_traces = np.dot(traces, theta_diff_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
        theta_diff_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_diff_conv)

        theta_off_traces = np.dot(traces, theta_off_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
        theta_off_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_off_conv)
        
        

        ax_anatomy2[i-count,0].scatter(coords[:, 1], coords[:, 2], c=theta_wrp_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,0].spines['right'].set_visible(False)
        ax_anatomy2[i-count,0].spines['top'].set_visible(False)
        ax_anatomy2[i-count,0].spines['left'].set_visible(False)
        ax_anatomy2[i-count,0].spines['bottom'].set_visible(False)
        ax_anatomy2[i-count,0].set_xticks([])
        ax_anatomy2[i-count,0].set_yticks([])
        ax_anatomy2[i-count,0].set_ylabel(fish_id)
        
        ax_anatomy2[i-count,1].scatter(coords[:, 1], coords[:, 2], c=theta_diff_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,1].axis('off')

        ax_anatomy2[i-count,2].scatter(coords[:, 1], coords[:, 2], c=theta_off_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,2].axis('off')
        
        ax_anatomy2[i-count,3].scatter(coords[selected, 1], coords[selected, 2], c=theta_wrp_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,3].axis('off')
        
        ax_anatomy2[i-count,4].scatter(coords[selected, 1], coords[selected, 2], c=theta_diff_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,4].axis('off')

        ax_anatomy2[i-count,5].scatter(coords[selected, 1], coords[selected, 2], c=theta_off_traces[selected], cmap='coolwarm', s=3, vmin=-1, vmax=1)
        ax_anatomy2[i-count,5].axis('off')
        
    except:
        print("no selected")
        count += 1

ax_anatomy1[0].plot(stim_t_conv, theta_wrp_conv)
ax_anatomy1[0].plot(stim_t_conv, theta_diff_conv + 80)
ax_anatomy1[0].plot(beh_log.t, beh_log.tail_sum*10 - 80)
ax_anatomy1[0].plot(stim_t_conv, theta_off_conv/5 - 200)
ax_anatomy1[0].set_xlim(0, stim_t_conv[-1])
ax_anatomy1[0].axis('off')
        
ax_anatomy2[0,0].set_title('Theta position')
ax_anatomy2[0,1].set_title('Theta diff')
ax_anatomy2[0,2].set_title('Theta off')

ax_anatomy2[0,3].set_title('Theta position - selected')
ax_anatomy2[0,4].set_title('Theta diff - selected')
ax_anatomy2[0,5].set_title('Theta off - selected')

file_name = 'theta_regressors_rois_clol.jpg'
fig_anatomy2.savefig(master / file_name, dpi=300)

file_name = 'theta_regressors_clol.jpg'
fig_anatomy1.savefig(master / file_name, dpi=300)

In [ ]:
# making one big figure:
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files_natmov = list(master.glob("*/*_f*_natmov"))
files_clol = list(master.glob("*/*_f*_clol"))

all_rois_clol = 0
num_fish_clol = 0
for i in range(len(files_clol)):
    path = files_clol[i]
    try:
        selected = fl.load(path / "selected.h5")
        traces = fl.load(path / "filtered_traces.h5", "/detr").T
        
        if all_rois_clol is 0:
            all_rois_clol = traces
        else:
            all_rois_clol = np.append(all_rois_clol, traces, axis=0)
        num_fish_clol += 1
    except:
        print("no selected")

print(num_fish_clol)
'''
all_rois_natmov = 0
num_fish_natmov = 0
for i in range(len(files_natmov)):
    path = files_natmov[i]
    try:
        selected = fl.load(path / "selected.h5")
        traces = fl.load(path / "filtered_traces.h5", "/detr").T
        print(np.shape(traces))
        
        if all_rois_natmov is 0:
            all_rois_natmov = traces
        else:
            print(i)
            all_rois_natmov = np.append(all_rois_natmov, traces, axis=0)
        num_fish_natmov += 1
    except:
        print("no selected")

print(num_fish_nclol)
'''


In [ ]:
np.shape(all_rois_clol)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def cluster_id_search(tree):
    nodes_list = []
    if tree.is_leaf():
        nodes_list.append(tree.get_id())
    else:
        nodes_list += cluster_id_search(tree.get_left())
        nodes_list += cluster_id_search(tree.get_right())
        
    return nodes_list

def find_trunc_dendro_clusters(linkage_mat, dendro):
    tree, branches = to_tree(linkage_mat, rd =True)
    ids = np.empty(linkage_mat.shape[0]+1, dtype=int)
    
    for i, clust in enumerate(dendro["leaves"]):
        ids[cluster_id_search(branches[clust])] = i
        
    return ids

In [ ]:
# clustering of traces to see if the responses are more motor (fish specific) or visual (similar across fish):

fig1 = plt.figure(figsize=(10,7))
linked = linkage(all_rois_clol, method='ward')
dend = dendrogram(linked)

cluster = AgglomerativeClustering(n_clusters=12, affinity='euclidean', linkage='ward')
her_clustering = cluster.fit_predict(all_rois_clol)


In [ ]:
# make truncated tree to get clusters ids. 
# Ugly but necessary to get the same sequence of leaves as the cut.
plt.figure(figsize=(0.1, 0.1))  
n_clust = 8
dendro = dendrogram(linked, n_clust, truncate_mode ="lastp")
plt.close()
cluster_ids = dendro["leaves"]
labels = find_trunc_dendro_clusters(linked, dendro)

In [ ]:
meanresps = all_rois_clol
base_sub_mean = (meanresps.T - np.nanmean(meanresps[:,:8], 1)).T
X = base_sub_mean

In [ ]:
from skimage import color
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, cut_tree, set_link_color_palette
#import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm

In [ ]:
def shade_plot(stim, ax=None, gamma=1/6, shade_range=(0.6, 0.9)):
    if type(stim) == list:  # these would be transitions
        _shade_plot(stim, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == Data:  # fish data
        transitions = find_transitions(Data.resampled_stim, Data.time_im_rep)
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == np.ndarray:  # stimulus array
        transitions = find_transitions(stim[:,1], stim[:,0])
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == tuple:  # time, lum tuple
        transitions = find_transitions(stim[1], stim[0])
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)


def _shade_plot(lum_transitions, ax=None, gamma=1/6, shade_range=(0.6, 0.9)):

    if ax is None:
        ax = plt.gca()
    shade = lum_transitions[0][1]
    for i in range(len(lum_transitions)-1):
        shade = shade + lum_transitions[i][1]
        new_shade = shade_range[0] + np.power(np.abs(shade), gamma) * (shade_range[1] - shade_range[0])
        ax.axvspan(lum_transitions[i][0], lum_transitions[i+1][0], color=(new_shade, )*3)
        

def _find_thr(linked, n_clust):
    interval = [0, 2000]
    new_height = np.mean(interval)
    clust = 0
    n_clust = n_clust
    while clust != n_clust:
        new_height = np.mean(interval)
        clust = cut_tree(linked, height=new_height).max()
        if clust > n_clust:
            interval[0] = new_height
        elif clust < n_clust:
            interval[1] = new_height


    return new_height


def find_plot_thr(linked, n_clust):
    min_thr = _find_thr(linked, n_clust - 1)
    return min_thr  


In [ ]:
def plot_clusters_dendro(traces, stim, linkage_mat, labels, dendrolims=(900, 30),
                         thr=None, f_lim=1.5, gamma=1):
    fig_clust, ax = plt.subplots(3, 1, figsize=(10, 10))
    hexac = cluster_cols()

    n_clust = labels.max() + 1

    ##################
    ### Dendrogram ###
    # Compute and plot first dendrogram.
    if thr is None:
        thr = find_plot_thr(linkage_mat, n_clust)

    set_link_color_palette(hexac)
    ax_traces = plt.subplot2grid((2, 2), (0, 0))
    ax_clusters = plt.subplot2grid((2, 2), (0, 1))
    ax_dendro = plt.subplot2grid((2, 2), (1, 0), colspan=2)

    #ax_dendro = ax[2]
    #ax_traces = ax[1]
    #ax_clusters = ax[0]
    
    panel_dendro = dendrogram(linkage_mat,
                              color_threshold=thr,
                              #orientation='left',
                              distance_sort='descending',
                              show_leaf_counts=False,
                              no_labels=True,
                              above_threshold_color='#%02x%02x%02x' % (
                              120, 120, 120))
    
    ax_dendro.axhline(thr, linewidth=0.7, color="k")
    ax_dendro.axis("off")

    # Plot traces matrix.
    im = ax_traces.imshow(traces[panel_dendro["leaves"], :],
                         aspect='auto', origin='lower', cmap=cm.RdBu_r,
                         vmin=-f_lim, vmax=f_lim)
    ax_traces.axes.spines['left'].set_visible(False)
    ax_traces.set_yticks([])

    # Time bar:
    dt = stim[1, 0]
    barlength = 10
    bounds = np.array([traces.shape[1] - barlength / dt,
                       traces.shape[1]])

    ##################
    # Cluster sizes ##
    # Calculate size of each defined cluster to put colored labels on the side.
    # Find indervals spanned by each cluster in the sorted traces matrix.
    # Add percentages spanned by each cluster.
    sizes = np.cumsum(np.array([np.sum(labels == i) for i in range(np.max(labels) + 1)]))
    intervals = np.insert(sizes, 0, 0)

    ##################
    # Cluster means ##

    for i in range(n_clust):
        ax_clusters.plot(np.nanmean(traces[labels == i, :], 0) +
                      i * 5, label=i, color=hexac[i])
    ax_clusters.axes.spines['left'].set_visible(False)
    ax_clusters.set_yticks([])

    barlength = 10
    ax_traces.axis("off")
    ax_clusters.axis("off")

    return fig_clust, ax_clusters

def cluster_cols():
    # color_list = ["lightblue", "lightcoral", "orange", "springgreen", "deepskyblue", "mediumpurple","gold", "cyan", "crimson", "deeppink", "lawngreen", "darkviolet", "orchid", "limegreen", "seagreen", "chocolate", "blue", "navy"]
    color_list = ["#cc566a", "#cd6c39", "#a39440", "#64ac48", "#4aac8d", "#688bcd", "#8562cc", "#c361aa"]
    # color_list = ["#ff5c67", "#af0006", "#ffa468", "#8c5f00", "#e4a400", "#d5c86f", "#939400", "#a7d380", "#138b00", "#42e087", "#00a86d", "#81c7a8", "#019a82", "#1eaaff", "#0268bb", "#5951d7", "#6b4570", "#ad20aa", "#ffa1e2", "#ff4a94"]
    return color_list

In [ ]:
stim = np.asarray([[1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]])
fig_clust, ax_clust = plot_clusters_dendro(all_rois_clol, stim, linked, labels)#, dendrolims=(940, 0))

f = files_clol[0]
exp = LotrExperiment(path=f)
    
plt.show()
file_name = 'hierarchical_clustering_220531.jpg'
fig_clust.savefig(str(master/file_name), dpi=300)

In [ ]:
### getting the indices for each fish:
num_fish = 7
fish_inds = np.zeros(num_fish)
print(num_fish)
num_rois = np.zeros(num_fish)
count = 0
for i in range(len(files_clol)):
    f = files_clol[i]
    traces = fl.load(f / "filtered_traces.h5", "/detr").T
    num_traces = np.shape(traces)[0]
    try:
        selected = fl.load(f / "selected.h5")
        num_rois[i-count] = num_traces 
        fish_inds[i-count] = i
    except:
        count += 1
print(num_rois)
print(fish_inds)

In [ ]:
ind2 = np.cumsum(num_rois).astype(int)
ind2

In [ ]:
num_rois= num_rois.astype(int)

In [ ]:
ind1 = np.zeros(num_fish+1, dtype=int)
for i in range(num_fish):
    ind1[i+1] = ind2[i]
ind1

In [ ]:
np.shape(labels)
np.shape(coords)

In [ ]:
num_row = 3
num_col = 3
fig1, ax1 = plt.subplots(num_row, num_col, figsize=(8, 8))

for i in range(num_row*num_row):
    r = i // num_row
    c = np.mod(i, num_row)
    
    try:
        path = files_clol[fish_inds[i].astype(int)]
        coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
        col_hrc = labels[ind1[i]:ind2[i]]

        ax1[r, c].scatter(coords[:, 1], coords[:, 2], c=col_hrc, cmap='rainbow', s=3, vmin=0, vmax=7)
        ax1[r, c].axis('off')
    except:
        print(i)
        ax1[r, c].axis('off')

    
plt.show()
file_name = 'clusters_hrc_rois_220531.jpg'
fig1.savefig(str(master/file_name), dpi=300)

In [ ]:
num_row = 3
num_col = 3
fig1, ax1 = plt.subplots(num_row, num_col, figsize=(8, 8))

for i in range(num_row*num_row):
    r = i // num_row
    c = np.mod(i, num_row)
    
    try:
        path = files_clol[fish_inds[i].astype(int)]
        coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
        selected = fl.load(path / "selected.h5")
        col_hrc = labels[ind1[i]:ind2[i]]
        
        ax1[r, c].scatter(coords[:, 1], coords[:, 2], c='gray', s=3)
        ax1[r, c].scatter(coords[selected, 1], coords[selected, 2], c=col_hrc[selected], cmap='rainbow', s=3, vmin=0, vmax=7)
        ax1[r, c].axis('off')
    except:
        ax1[r, c].axis('off')

    
plt.show()
file_name = 'clusters_hrc_rois_selected_220531.jpg'
fig1.savefig(str(master/file_name), dpi=300)